In [10]:
import os
import numpy as np
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')
import scipy.optimize as opt
import tensorflow as tf
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
import keras
import pandas as pd
gpus = tf.config.experimental.list_physical_devices('GPU')


In [14]:
import matplotlib.pyplot as plt
import h5py
import sys
from keras import metrics

from tqdm import tqdm
from sklearn.metrics import roc_curve, auc

# Layers

In [7]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        
        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)
        
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)
        
        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
        return output, attention_weights
    
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.
    
    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
    to (..., seq_len_q, seq_len_k). Defaults to None.
    
    Returns:
    output, attention_weights
    """
    
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    
    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    
    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  
    
    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)
    
    return output, attention_weights

# Load your data

In [5]:
## The shape of input data like (Number of data, x-axis, y-axis)
## The meaning of data sets (Number of data, sentens, words)
rinv = "0p3"
file_path = "pre_LL_"+rinv+".h5"
hdf_file = '/local_disk2/james/Taylor_data/semi-visible-jets-ml/data/LL/LL-0p3.h5'
hf = h5py.File(hdf_file, "r")
X = hf["features"][:]
Y = hf["targets"][:]
X.shape # (number of jet images, x-axis, y-axis )

(1900000, 32, 32)

In [6]:
N = len(X)
Ntrain, Nval, Ntest = int(N/5*4),  int(N/10),  int(N/10)
Xim_train, Xim_val, Xim_test = X[:Ntrain], X[Ntrain:Nval+Ntrain], X[Nval+Ntrain:N]
yim_train, yim_val, yim_test = Y[:Ntrain], Y[Ntrain:Nval+Ntrain], Y[Nval+Ntrain:N]

# Model Architecture

In [8]:
d_mode = X.shape[-1]
dff = 2048
num_heads =  X.shape[-1]
dprate = 0.1
R = 1.65
inputs = tf.keras.Input(shape=( X.shape[-2], X.shape[-1]))

x = inputs
print(x.shape)

# x = x+R
# print(x.shape)



# x = tf.expand_dims(x, axis=-1)
# print(x.shape)
# x = Attention()(x)
# print(x.shape)

# x = Linear()(x)
# print(x.shape)

##---------------------------------------Encode layer
print('att')
att, _ = MultiHeadAttention(d_mode, num_heads)(x, x, x, None)
print(att.shape)

# print('att')
# att = point_wise_feed_forward_network(d_mode, dff)(att)
# print(att.shape)

print('att')
att = tf.keras.layers.Dropout(dprate)(att, training=False)
print(att.shape)

x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x+att)
print(x.shape)

att = point_wise_feed_forward_network(d_mode, dff)(x)
print(att.shape)

att = tf.keras.layers.Dropout(dprate)(att, training=False)
print(att.shape)

x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x+att)
print(x.shape)
##------------------------------------------------------------------------

##----------------------------------------------- Decode layer
att, _ = MultiHeadAttention(d_mode, num_heads)(x, x, x, None)
print(att.shape)
att = tf.keras.layers.Dropout(dprate)(att, training=False)
print(att.shape)
x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x+att)
print(x.shape)

att, _ = MultiHeadAttention(d_mode, num_heads)(x, x, x, None)
print(att.shape)
att = tf.keras.layers.Dropout(dprate)(att, training=False)
print(att.shape)
x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x+att)
print(x.shape)

att = point_wise_feed_forward_network(d_mode, dff)(x)
print(att.shape)
att = tf.keras.layers.Dropout(dprate)(att, training=False)
print(att.shape)   
x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x+att)
print(x.shape)

   
##------------------------------------------------------------------------
x = tf.keras.layers.Flatten()(x)
print(x.shape)

# x = tf.keras.layers.Dense(256)(x)
# print(x.shape)

# x= tf.keras.layers.Dropout(0.2)(x)
# print(x.shape)

# x = tf.keras.layers.Dense(128)(x)
# print(x.shape)

# x= tf.keras.layers.Dropout(0.2)(x)
# print(x.shape)

x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
print(x.shape)





modelTransformer = tf.keras.Model(inputs= inputs, outputs=x, name='Transformer')


# modelTransformer = tf.keras.Model(inputs= inputs, outputs=x, name='Transformer')


(None, 32, 32)
att
(None, None, 32)
att
(None, None, 32)
(None, 32, 32)
(None, 32, 32)
(None, 32, 32)
(None, 32, 32)
(None, None, 32)
(None, None, 32)
(None, 32, 32)
(None, None, 32)
(None, None, 32)
(None, 32, 32)
(None, 32, 32)
(None, 32, 32)
(None, 32, 32)
(None, 1024)
(None, 1)


In [11]:
model_type = "Transformer"
save_dir = './'
model_name = '%s_model_'% model_type +rinv 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)
#                                              monitor='val_auc',
#                                              mode='max',
#  monitor='val_acc',

# lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)

# progress_bar = keras.callbacks.ProgbarLogger()

# csv_logger = keras.callbacks.CSVLogger(save_dir+'CNN'+rinv+'.csv')
# csv_logger = keras.callbacks.CSVLogger(save_dir+'CNN_'+rinv+'_'+optn+'_filter.csv')
csv_logger = keras.callbacks.CSVLogger(save_dir+model_type+rinv+'.csv')


earlystop = keras.callbacks.EarlyStopping(
                            monitor="val_loss",
                            min_delta=1e-4,
                            patience=3, # 10
                            verbose=1,
                            mode='min', baseline=None, ## 'min' 
                            restore_best_weights=True)
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=2, min_lr=0.00001)
callbacks = [checkpoint, csv_logger,  earlystop ]


In [12]:
optimizer = tf.keras.optimizers.Adam()

In [15]:
modelTransformer.compile(optimizer=optimizer , loss="binary_crossentropy", metrics=['accuracy', metrics.AUC(name="auc")])
modelTransformer.summary()

Model: "Transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32)]     0                                            
__________________________________________________________________________________________________
multi_head_attention (MultiHead ((None, None, 32), ( 4224        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, None, 32)     0           multi_head_attention[0][0]       
__________________________________________________________________________________________________
tf_op_layer_AddV2 (TensorFlowOp [(None, 32, 32)]     0           input_1[0][0]                    
                                                                 dropout[0][0]          

# Training

In [ ]:
modelTransformer.fit(Xim_train, yim_train , validation_data=(Xim_val, yim_val), callbacks = callbacks, shuffle=True , epochs=400, batch_size=256, verbose=1)

1668/5938 [=======>......................] - ETA: 12:43 - loss: 0.5898 - accuracy: 0.6846 - auc: 0.7509

In [ ]:
modelTransformer.save("./Transformer_"+rinv)


# Plot

## ROC curve

In [ ]:
y_score = modelTransformer.predict(Xim_test)[:,0]
y_score = np.hstack(y_score)
# test=[i[1] for i in yim_test]
fpr , tpr , thresholds = roc_curve ( yim_test , y_score)
roc_auc = auc(tpr,fpr )
print("The area under the curves are:")
print("AUC:{0:.9f}".format(roc_auc))
if roc_auc<0.5:
    a = tpr
    tpr = fpr
    fpr = a
    roc_auc = 1 - roc_auc
    print("AUC:{0:.9f}".format(roc_auc))
    
# FalsePositiveFull, TruePositiveFull, ThresholdFull = metrics.roc_curve(y_test,Predictions)
plt.figure(figsize=(8,8))

plt.plot(tpr,fpr, label='Fully supervised: AUC={0:.5f}'.format(roc_auc))
plt.ylabel('False Positive Rate',fontsize=20)
plt.xlabel('True Positive Rate',fontsize=20)
plt.plot([0, 1], [0, 1], 'k--')
# plt.legend()
# plt.legend(bbox_to_anchor=(0.8, -0.17),ncol=2)

plt.legend(prop={'size': 14})



plt.savefig("./Transformer_"+rinv+"_roc.png")

# hf = h5py.File("./Keras_Tunner/PR_all_"+rinv+"_"+optn+"_filter_preprocessing.h5", 'w')
hf = h5py.File("./Transformer_"+rinv+".h5", 'w')

hf.create_dataset('fpr', data=fpr)
hf.create_dataset('tpr', data=tpr)
hf.close()


plt.show()

## Learning curve

In [ ]:
plt.figure(figsize=(8,8))
# LOSS = pd.read_csv(save_dir+"CNN_"+rinv+"_"+optn+"_filter.csv")
# LOSS = pd.read_csv(save_dir+"CNN_all_"+rinv+"_"+optn+"_filter_preprocessing.csv")
LOSS = pd.read_csv(save_dir+model_type+rinv+'.csv')


plt.title("Learning Curve")
plt.plot(LOSS["loss"], label='loss',c='blue')
plt.plot(LOSS["val_loss"], label='val_loss',c='red')
plt.plot(LOSS["accuracy"], linestyle='--', label='accuracy',c='blue')
plt.plot(LOSS["val_accuracy"], linestyle='--', label='val_accuracy',c='red')

plt.plot(LOSS["auc"], linestyle=':', label='auc',c='blue')
plt.plot(LOSS["val_auc"], linestyle=':', label='val_auc',c='red')
# plt.plot([0, 1], [1, 1], '--')

# plt.ylim([0.3,1])
plt.ylabel('loss',fontsize=20)
plt.xlabel('epoch',fontsize=20)
# plt.legend(bbox_to_anchor=(0.8, -0.17),ncol=2)
plt.legend(bbox_to_anchor=(0.8, -0.17),ncol=2)
plt.tight_layout()
# plt.savefig("./Keras_Tunner/CNN_"+rinv+"_"+optn+"_filter_loss.png")
# plt.savefig("./Keras_Tunner/CNN_all_"+rinv+"_"+optn+"_filter_loss_preprocessing.png")
plt.savefig("./Transformer_"+rinv+"_loss.png")



plt.show()